## Загрузка данных и подключение библиотек

In [1]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c tabular-playground-series-may-2021
!pip install catboost
!pip install eli5
!pip install optuna
!pip install shap
!pip install scikit-learn-extra
!unzip /content/test.csv.zip
!unzip /content/train.csv.zip
!unzip /content/sample_submission.csv.zip

kaggle.json
  0% 0.00/128k [00:00<?, ?B/s]
100% 128k/128k [00:00<00:00, 40.2MB/s]
  0% 0.00/851k [00:00<?, ?B/s]
100% 851k/851k [00:00<00:00, 54.9MB/s]
  0% 0.00/1.72M [00:00<?, ?B/s]
100% 1.72M/1.72M [00:00<00:00, 117MB/s]
     |████████████████████████████████| 67.3MB 42kB/s 
     |████████████████████████████████| 112kB 3.0MB/s 
     |████████████████████████████████| 296kB 2.9MB/s 
     |████████████████████████████████| 174kB 12.7MB/s 
     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 112kB 11.8MB/s 
     |████████████████████████████████| 143kB 12.4MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=15bbcf95d182d88114d267c6e63d3af31c8f1238394d62c015ca792dcf31dee2
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully buil

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import catboost as cb
import sklearn
import xgboost as xgb
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA, FactorAnalysis as FA
from typing import List, Optional
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, train_test_split
from typing import List, Tuple
import scipy.stats as ss
from sklearn_extra.cluster import KMedoids
import math
from sklearn.utils.validation import check_is_fitted
import eli5
from sklearn.base import BaseEstimator, TransformerMixin
import time
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import shap
import missingno as msno
from sklearn.inspection import permutation_importance
from eli5.sklearn import PermutationImportance
import optuna
from sklearn.metrics import log_loss
pd.plotting.register_matplotlib_converters()
%matplotlib inline
sns.set(color_codes=True)
pal = sns.color_palette("viridis", 10)
sns.set_palette(pal)

## Используемые функции

In [2]:
def get_input(data_path: str) -> pd.DataFrame:
  """
  Считывание данных и вывод основной информации о наборе данных.

  Parmeters
  ---------
  data_path: str - название файла

  Returns
  -------
  data: pandas.core.frame.DataFrame - загруженный набор данных в pandas.Dataframe
  """
  base_path = "/content"
  data = pd.read_csv(f"{base_path}/{data_path}")
  data.columns = [col.lower() for col in data.columns]
  print(f"{data_path}: shape = {data.shape[0]} rows, {data.shape[1]} cols")
  return data

In [3]:
def plot_feature_importance(importance, names, model_type, figsize=(10,8)):

  #Create arrays from feature importance and feature names
  feature_importance = np.array(importance)
  feature_names = np.array(names)

  #Create a DataFrame using a Dictionary
  data={'feature_names':feature_names,'feature_importance':feature_importance}
  fi_df = pd.DataFrame(data)

  #Sort the DataFrame in order decreasing feature importance
  fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

  #Define size of bar plot
  plt.figure(figsize=figsize)
  #Plot Searborn bar chart
  sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
  #Add chart labels
  plt.title(model_type + ' FEATURE IMPORTANCE')
  plt.xlabel('FEATURE IMPORTANCE')
  plt.ylabel('FEATURE NAMES')
  return list(fi_df['feature_names'].values)

In [4]:
def multi_estimators_predict(estimators: List,
                             x_valid: pd.DataFrame,
                             y_valid = None,
                             metric: callable = None,
                             scalers: List = None):
  preds = []
  evals = []
  if scalers:
    assert len(estimators) == len(scalers)

  for i in range(len(estimators)):
    if scalers:
      pred = estimators[i].predict_proba(scalers[i].transform(x_valid))
    else:
      if type(estimators[0]) == xgb.core.Booster:
        pred = estimators[i].predict(x_valid)
      else:
        pred = estimators[i].predict_proba(x_valid)

    
    preds.append(pred)
    if (y_valid is not None) and (metric is not None):
      eval = metric(y_valid, pred)
      evals.append(eval)
  
  result = np.stack([preds], axis=0)
  result = np.squeeze(result, axis=0)
  result = np.mean(result, axis=0)

  if (y_valid is not None) & (metric is not None):
    for i, eval in enumerate(evals):
      print(f"Model {i} metric: {eval:.7}")
    print(f"Result model metric: {metric(y_valid, result):.7}")

  return result

In [5]:
def logistic_cv_fit(params, X, y, cv, scale=False):
    """
    Кросс-валидация для модели LogisticRegression.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признаков для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """

    estimators, folds_scores, scalers = [], [], []
    oof_preds = np.zeros((X.shape[0], 4))
    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]
        if scale:
          scaler = StandardScaler()
          x_train = scaler.fit_transform(x_train)
          x_valid = scaler.transform(x_valid)
        model = LogisticRegression(**params)
        model.fit(x_train, y_train)
        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)
        if scale:
          scalers.append(scaler)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds, scalers

In [6]:
def catboost_cv_fit(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели catboost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    if not categorical:
        categorical = "auto"

    estimators, folds_scores = [], []
    oof_preds = np.zeros((X.shape[0], 4))
    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = cb.CatBoostClassifier(**params)
        model.fit(
            x_train, y_train,
            #eval_set=[(x_valid, y_valid)], 
            verbose=10, 
            #early_stopping_rounds=100,
            #cat_features=categorical
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds

In [7]:
def lightgbm_cv_fit(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели lightgbm.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    if not categorical:
        categorical = "auto"

    estimators, folds_scores = [], []
    oof_preds = np.zeros((X.shape[0], 4))
    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = lgb.LGBMClassifier(**params)
        model.fit(
            x_train, y_train,
            #eval_set=[(x_valid, y_valid)],
            #eval_metric="multi_logloss", 
            verbose=10, 
            #early_stopping_rounds=50,
            #categorical_feature=categorical
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds

In [8]:
def xgboost_cv_fit(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели xgboost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    encoders: dict
        Список с объектами LabelEncoders.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    estimators, encoders = [], {}
    oof_preds = np.zeros((X.shape[0], 4))

    if categorical:
        for feature in categorical:
            encoder = LabelEncoder()
            X[feature] = encoder.fit_transform(X[feature].astype("str").fillna("NA"))
            encoders[feature] = encoder

    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):

        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = xgb.XGBClassifier(**params)
        model.fit(
            x_train, y_train,
            #eval_set=[(x_valid, y_valid)],
            #eval_metric="mlogloss", 
            verbose=10, 
            #early_stopping_rounds=50,
        )

        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        estimators.append(model)

    return estimators, encoders, oof_preds

## Загрузка данных и построение моделей

### lightgbm + optuna

In [ ]:
def objective(trial):
    data = get_input("train.csv")
    data.drop(columns='id', inplace=True)
    mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
    data['target'] = data['target'].map(mapper)
    target = data['target']
    data = data.drop(columns=['target'])

    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.5)
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        #"n_estimators": trial.suggest_int("n_estimators", 200, 800),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        'num_class': 4,
        'max_depth': trial.suggest_int('max_depth', 3,7),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.05, 0.13),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "multi_logloss")
    gbm = lgb.train(
        param, dtrain, 
        valid_sets=[dvalid], 
        verbose_eval=False, 
        callbacks=[pruning_callback]
    )

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)

    log_loss = sklearn.metrics.log_loss(valid_y, preds)
    return log_loss

In [ ]:
# Лучше подбирать гиперпараметры на всей data через кросс валидацию
import optuna.integration.lightgbm as lgb
def objective(trial):
    data = get_input("train.csv")
    data.drop(columns='id', inplace=True)
    mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
    data['target'] = data['target'].map(mapper)
    target = data['target']
    data = data.drop(columns=['target'])

    dtrain = lgb.Dataset(data, label=target)

    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        'num_class': 4,

        'max_depth': trial.suggest_int('max_depth', 3,7),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.03, 0.13 ),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "multi_logloss")
    result = lgb.cv(
      params=param,
      train_set=dtrain,
      num_boost_round=10000,
      early_stopping_rounds=100,
      #verbose_eval=10,
      stratified=True,
      seed=42,
      metrics="multi_logloss",
      shuffle=True,
      nfold=3
    )
 
    log_loss = result['multi_logloss-mean'][-1] + result['multi_logloss-stdv'][-1]
    return log_loss

In [ ]:
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=20), direction="minimize"
    )
study.optimize(objective, n_trials=200)

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

[I 2021-05-25 20:05:15,460] A new study created in memory with name: no-name-67bb9b06-6056-4c5f-9440-69bfd5220d67


train.csv: shape = 100000 rows, 52 cols


[I 2021-05-25 20:06:07,912] Trial 0 finished with value: 1.095420663265998 and parameters: {'lambda_l1': 0.00024587192153438774, 'lambda_l2': 2.9664723207282242e-08, 'num_leaves': 223, 'feature_fraction': 0.7099892913219672, 'bagging_fraction': 0.7228809986014417, 'bagging_freq': 6, 'min_child_samples': 97, 'max_depth': 5, 'learning_rate': 0.08549446305776937}. Best is trial 0 with value: 1.095420663265998.


train.csv: shape = 100000 rows, 52 cols


KeyboardInterrupt: ignored

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
print(study.best_params)

{'lambda_l1': 1.45120566383297e-06, 'lambda_l2': 0.003368552965821498, 'num_leaves': 32, 'feature_fraction': 0.4471131328810426, 'bagging_fraction': 0.5888198372400193, 'bagging_freq': 4, 'min_child_samples': 32}


In [ ]:
study.best_value

1.0833801288748757

In [ ]:
data = get_input("train.csv")
data.drop(columns='id', inplace=True)
mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
data['target'] = data['target'].map(mapper)
target = data['target']
data = data.drop(columns=['target'])

lgb_params = {
    "boosting_type ": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    #'num_boost_round': 100,
    'num_class': 4,
    'lambda_l1': 0.02259438646302076,
    'lambda_l2': 3.3834082296901886e-05,
    'num_leaves': 256,
    'feature_fraction': 0.42939073275678896,
    'bagging_fraction': 0.8760623722003144,
    'bagging_freq': 4,
    'min_child_samples': 71,
    'max_depth': 4,
    'learning_rate': 0.07658957460133804,
}

dtrain = lgb.Dataset(data=data, label=target)

result = lgb.cv(
    params=lgb_params,
    train_set=dtrain,
    num_boost_round=2000,
    early_stopping_rounds=100,
    verbose_eval=10,
    stratified=True,
    seed=42,
    metrics="multi_logloss",
    shuffle=True,
    nfold=5
)

train.csv: shape = 100000 rows, 52 cols
[10]	cv_agg's multi_logloss: 1.11339 + 0.00019763
[20]	cv_agg's multi_logloss: 1.10998 + 0.000341567
[30]	cv_agg's multi_logloss: 1.10748 + 0.000399434
[40]	cv_agg's multi_logloss: 1.1054 + 0.000482044
[50]	cv_agg's multi_logloss: 1.10365 + 0.000591791
[60]	cv_agg's multi_logloss: 1.10218 + 0.000701451
[70]	cv_agg's multi_logloss: 1.10096 + 0.000759168
[80]	cv_agg's multi_logloss: 1.09988 + 0.000727296
[90]	cv_agg's multi_logloss: 1.09888 + 0.000727316
[100]	cv_agg's multi_logloss: 1.09803 + 0.000788178
[110]	cv_agg's multi_logloss: 1.09726 + 0.00084155
[120]	cv_agg's multi_logloss: 1.09667 + 0.000892679
[130]	cv_agg's multi_logloss: 1.09616 + 0.000970538
[140]	cv_agg's multi_logloss: 1.09569 + 0.00102404
[150]	cv_agg's multi_logloss: 1.09526 + 0.00106349
[160]	cv_agg's multi_logloss: 1.09494 + 0.0011131
[170]	cv_agg's multi_logloss: 1.09459 + 0.00114867
[180]	cv_agg's multi_logloss: 1.09433 + 0.00113994
[190]	cv_agg's multi_logloss: 1.09407 + 0.

In [ ]:
len(result['multi_logloss-mean'])

334

In [11]:
import lightgbm as lgb
data = get_input("train.csv")
test = get_input("test.csv")

data.drop(columns='id', inplace=True)
mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
data['target'] = data['target'].map(mapper)

target = data['target']
data = data.drop(columns=['target'])

#train, valid = train_test_split(
#    data, train_size=0.7, shuffle=True, random_state=1,
#)

#train.reset_index(inplace=True, drop=True)
#valid.reset_index(inplace=True, drop=True)

#y_train = train['target']
#x_train = train.drop(columns=['target'])
#y_valid = valid['target']
#x_valid = valid.drop(columns=['target'])

train.csv: shape = 100000 rows, 52 cols
test.csv: shape = 50000 rows, 51 cols


In [15]:
lgb_params = {
    "boosting_type ": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    'n_estimators': 500,
    'num_class': 4,
    'lambda_l1': 0.02259438646302076,
    'lambda_l2': 3.3834082296901886e-05,
    'num_leaves': 256,
    'feature_fraction': 0.42939073275678896,
    'bagging_fraction': 0.8760623722003144,
    'bagging_freq': 4,
    'min_child_samples': 71,
    'max_depth': 3,
    'learning_rate': 0.07658957460133804,
    'random_state': 42,
}


lgb_params = {
    "boosting_type ": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    'n_estimators': 710,
    'num_class': 4,
    'lambda_l1': 0.02,
    'lambda_l2': 150,
    'num_leaves': 7,
    'feature_fraction': 0.11,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    'min_child_samples': 80,
    'max_depth': 5,
    'learning_rate': 0.075,
    'random_state': 42,
    #'max_bin': 50,
    #'device' : 'gpu'
}


cv = StratifiedKFold(n_splits=15, random_state=435, shuffle=True)

lgb_estimators, lgb_oof = lightgbm_cv_fit(
    lgb_params, data, target, cv, #categorical=categorical_feature_names
)

Sun May 30 13:25:22 2021, Cross-Validation, 100000 rows, 50 cols
Fold 1, Valid score = 1.09527
Fold 2, Valid score = 1.08856
Fold 3, Valid score = 1.08425
Fold 5, Valid score = 1.08976
Fold 6, Valid score = 1.093
Fold 7, Valid score = 1.09338
Fold 8, Valid score = 1.08778
Fold 9, Valid score = 1.08888
Fold 10, Valid score = 1.09407
Fold 11, Valid score = 1.08974
Fold 12, Valid score = 1.09084
Fold 13, Valid score = 1.09176
Fold 14, Valid score = 1.08488
Fold 15, Valid score = 1.08688
Score by each fold: [1.09527, 1.08856, 1.08425, 1.09275, 1.08976, 1.093, 1.09338, 1.08778, 1.08888, 1.09407, 1.08974, 1.09084, 1.09176, 1.08488, 1.08688]


In [17]:
print(f"Out of fold log loss {metrics.log_loss(target, lgb_oof)}")

Out of fold log loss 1.0901208836954104


In [ ]:
lgb_estimators[0]

LGBMClassifier(bagging_fraction=0.8760623722003144, bagging_freq=4,
               boosting_type ='gbdt', feature_fraction=0.42939073275678896,
               lambda_l1=0.02259438646302076, lambda_l2=3.3834082296901886e-05,
               learning_rate=0.07658957460133804, max_depth=4,
               metric='multi_logloss', min_child_samples=71, n_estimators=334,
               num_class=4, num_leaves=256, objective='multiclass',
               random_state=42)

### Catboost

In [ ]:
# optuna с кросс валидацией работает очень долго поэтому первое приближение получим 
# на отложенной выборке а потом дотюним
def objective(trial):
    data = get_input("train.csv")
    data.drop(columns='id', inplace=True)
    mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
    data['target'] = data['target'].map(mapper)
    target = data['target']
    data = data.drop(columns=['target'])

    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.35)

    param = {
        "loss_function": "MultiClass",
        "eval_metric": "MultiClass",
        "task_type": "GPU",
        
        #"colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        #"subsample": trial.suggest_float("subsample", 0.1, 1),

        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'depth': trial.suggest_int('depth', 3,7),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.13 ),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),                       
        #'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), # for bayesian bootstrap only

        #'bootstrap_type': 'Bernoulli',
        #'leaf_estimation_method': 'Newton',

        'grow_policy': "SymmetricTree",
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "multi_logloss")
    gbm = cb.CatBoostClassifier(**param)

    gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)
    preds = gbm.predict_proba(valid_x)


    log_loss = sklearn.metrics.log_loss(valid_y, preds)
    return log_loss

In [ ]:
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=20), direction="minimize"
    )
study.optimize(objective, n_trials=200)

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

[I 2021-05-25 20:07:18,768] A new study created in memory with name: no-name-234ea1e3-17bb-4364-985a-ec4112338ece


train.csv: shape = 100000 rows, 52 cols


[I 2021-05-25 20:07:19,978] Trial 0 finished with value: 1.186633703881519 and parameters: {'min_data_in_leaf': 24, 'depth': 3, 'learning_rate': 0.12078519555155853, 'random_strength': 58, 'l2_leaf_reg': 2.6581884406103866e-06}. Best is trial 0 with value: 1.186633703881519.


train.csv: shape = 100000 rows, 52 cols


[I 2021-05-25 20:07:25,654] Trial 1 finished with value: 1.0898680635762144 and parameters: {'min_data_in_leaf': 33, 'depth': 4, 'learning_rate': 0.07195112205603399, 'random_strength': 86, 'l2_leaf_reg': 1.0362623676141234}. Best is trial 1 with value: 1.0898680635762144.


...

train.csv: shape = 100000 rows, 52 cols


[I 2021-05-25 20:24:06,281] Trial 198 finished with value: 1.1032557383383232 and parameters: {'min_data_in_leaf': 80, 'depth': 5, 'learning_rate': 0.06550125070387704, 'random_strength': 83, 'l2_leaf_reg': 7.8067576913576975}. Best is trial 152 with value: 1.0836613774733865.


train.csv: shape = 100000 rows, 52 cols


[I 2021-05-25 20:24:13,011] Trial 199 finished with value: 1.0937628032274382 and parameters: {'min_data_in_leaf': 48, 'depth': 5, 'learning_rate': 0.03715741624813227, 'random_strength': 52, 'l2_leaf_reg': 3.6429876663953307}. Best is trial 152 with value: 1.0836613774733865.


Best trial:
  Params: 
    min_data_in_leaf: 43
    depth: 5
    learning_rate: 0.07928546561861827
    random_strength: 66
    l2_leaf_reg: 2.9448952422509165


In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
data = get_input("train.csv")
data.drop(columns='id', inplace=True)
mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
data['target'] = data['target'].map(mapper)
target = data['target']
data = data.drop(columns=['target'])

cb_params = {
        "loss_function": "MultiClass",
        "eval_metric": "MultiClass",
        "task_type": "GPU",
        'min_data_in_leaf': 25,
        'depth': 3,
        'learning_rate': 0.09,
        'random_strength' :10,
        "l2_leaf_reg": 100,                       
        'grow_policy': "SymmetricTree",
        #"thread_count": 10,
        "random_seed": 27,
    }
cv_result = cb.cv(
        pool = cb.Pool(data, target),
        params = cb_params,
        plot=True,
        shuffle = True,
        stratified = True,
        seed = 42,
        iterations = 10000,
        early_stopping_rounds = 100,
        fold_count = 5,
        as_pandas = True,
        verbose_eval = 10
        )

In [13]:
data = get_input("train.csv")
data.drop(columns='id', inplace=True)
mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
data['target'] = data['target'].map(mapper)
target = data['target']
data = data.drop(columns=['target'])

cb_params = {
    "n_estimators": 1700,
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    "task_type": "GPU",
    'min_data_in_leaf': 25,
    'depth': 3,
    'learning_rate': 0.09,
    'random_strength' : 10,
    "l2_leaf_reg": 100,                       
    'grow_policy': "SymmetricTree",
    "random_seed": 27,
}

cb_params = {
    "n_estimators": 4000,
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    "task_type": "GPU",
    'min_data_in_leaf': 35,
    'depth': 3,
    'learning_rate': 0.06,
    'random_strength' : 10,
    "l2_leaf_reg": 300,                       
    'grow_policy': "SymmetricTree",
    "random_seed": 27,
    "classes_count": 4,
    'bootstrap_type':  'Bayesian',
    #'bagging_temperature': 50, #for Bayesian bootstrap_type
    #'subsample': 0.6 # for Poisson, Bernoulli, MVS bootstrap_type
}

cv = StratifiedKFold(n_splits=15, random_state=435, shuffle=True)

cb_estimators, cb_oof = catboost_cv_fit(
    cb_params, data, target, cv, #categorical=categorical_feature_names
)

train.csv: shape = 100000 rows, 52 cols
Sun May 30 13:21:15 2021, Cross-Validation, 100000 rows, 50 cols


0:	learn: 1.3525660	total: 5.74ms	remaining: 22.9s
10:	learn: 1.1880350	total: 44.6ms	remaining: 16.2s
20:	learn: 1.1387308	total: 92.5ms	remaining: 17.5s
30:	learn: 1.1211052	total: 131ms	remaining: 16.8s
40:	learn: 1.1144463	total: 170ms	remaining: 16.4s
50:	learn: 1.1113674	total: 223ms	remaining: 17.3s
60:	learn: 1.1096463	total: 271ms	remaining: 17.5s
70:	learn: 1.1084172	total: 310ms	remaining: 17.1s
80:	learn: 1.1073541	total: 349ms	remaining: 16.9s
90:	learn: 1.1064302	total: 388ms	remaining: 16.7s
100:	learn: 1.1056999	total: 436ms	remaining: 16.8s
110:	learn: 1.1050359	total: 472ms	remaining: 16.5s
120:	learn: 1.1042278	total: 514ms	remaining: 16.5s
130:	learn: 1.1036936	total: 550ms	remaining: 16.2s
140:	learn: 1.1031845	total: 593ms	remaining: 16.2s
150:	learn: 1.1027435	total: 629ms	remaining: 16s
160:	learn: 1.1023729	total: 666ms	remaining: 15.9s
170:	learn: 1.1019932	total: 702ms	remaining: 15.7s
180:	learn: 1.1013862	total: 741ms	remaining: 15.6s
190:	learn: 1.1008386	

0:	learn: 1.3525662	total: 5.02ms	remaining: 20.1s
10:	learn: 1.1883377	total: 42.2ms	remaining: 15.3s
20:	learn: 1.1387009	total: 81.8ms	remaining: 15.5s
30:	learn: 1.1212334	total: 119ms	remaining: 15.2s
40:	learn: 1.1144277	total: 161ms	remaining: 15.6s
50:	learn: 1.1112639	total: 198ms	remaining: 15.3s
60:	learn: 1.1095575	total: 249ms	remaining: 16.1s
70:	learn: 1.1082230	total: 284ms	remaining: 15.7s
80:	learn: 1.1072930	total: 320ms	remaining: 15.5s
90:	learn: 1.1064053	total: 355ms	remaining: 15.2s
100:	learn: 1.1057273	total: 390ms	remaining: 15.1s
110:	learn: 1.1050816	total: 424ms	remaining: 14.9s
120:	learn: 1.1044835	total: 463ms	remaining: 14.8s
130:	learn: 1.1038780	total: 498ms	remaining: 14.7s
140:	learn: 1.1034747	total: 532ms	remaining: 14.6s
150:	learn: 1.1031131	total: 565ms	remaining: 14.4s
160:	learn: 1.1026648	total: 596ms	remaining: 14.2s
170:	learn: 1.1021226	total: 631ms	remaining: 14.1s
180:	learn: 1.1016646	total: 668ms	remaining: 14.1s
190:	learn: 1.101103

0:	learn: 1.3525757	total: 5.18ms	remaining: 20.7s
10:	learn: 1.1884396	total: 44.1ms	remaining: 16s
20:	learn: 1.1388986	total: 82.1ms	remaining: 15.6s
30:	learn: 1.1213731	total: 120ms	remaining: 15.4s
40:	learn: 1.1145534	total: 168ms	remaining: 16.2s
50:	learn: 1.1115165	total: 211ms	remaining: 16.3s
60:	learn: 1.1098832	total: 261ms	remaining: 16.9s
70:	learn: 1.1085309	total: 299ms	remaining: 16.6s
80:	learn: 1.1074879	total: 356ms	remaining: 17.2s
90:	learn: 1.1064720	total: 394ms	remaining: 16.9s
100:	learn: 1.1055901	total: 437ms	remaining: 16.9s
110:	learn: 1.1049636	total: 477ms	remaining: 16.7s
120:	learn: 1.1042366	total: 515ms	remaining: 16.5s
130:	learn: 1.1036838	total: 550ms	remaining: 16.2s
140:	learn: 1.1034074	total: 585ms	remaining: 16s
150:	learn: 1.1029754	total: 618ms	remaining: 15.7s
160:	learn: 1.1025640	total: 656ms	remaining: 15.6s
170:	learn: 1.1021307	total: 691ms	remaining: 15.5s
180:	learn: 1.1016592	total: 728ms	remaining: 15.4s
190:	learn: 1.1012203	to

0:	learn: 1.3525927	total: 5.44ms	remaining: 21.8s
10:	learn: 1.1883344	total: 44.3ms	remaining: 16.1s
20:	learn: 1.1387004	total: 85.2ms	remaining: 16.1s
30:	learn: 1.1210619	total: 126ms	remaining: 16.1s
40:	learn: 1.1143659	total: 173ms	remaining: 16.7s
50:	learn: 1.1112374	total: 231ms	remaining: 17.9s
60:	learn: 1.1095273	total: 274ms	remaining: 17.7s
70:	learn: 1.1081965	total: 314ms	remaining: 17.4s
80:	learn: 1.1071441	total: 355ms	remaining: 17.2s
90:	learn: 1.1061738	total: 393ms	remaining: 16.9s
100:	learn: 1.1053186	total: 438ms	remaining: 16.9s
110:	learn: 1.1046768	total: 475ms	remaining: 16.6s
120:	learn: 1.1039992	total: 515ms	remaining: 16.5s
130:	learn: 1.1034656	total: 550ms	remaining: 16.3s
140:	learn: 1.1031392	total: 587ms	remaining: 16.1s
150:	learn: 1.1026506	total: 627ms	remaining: 16s
160:	learn: 1.1023024	total: 669ms	remaining: 16s
170:	learn: 1.1018107	total: 709ms	remaining: 15.9s
180:	learn: 1.1013022	total: 751ms	remaining: 15.9s
190:	learn: 1.1008898	to

...

0:	learn: 1.3525642	total: 4.71ms	remaining: 18.9s
10:	learn: 1.1885605	total: 40.3ms	remaining: 14.6s
20:	learn: 1.1388795	total: 75.7ms	remaining: 14.3s
30:	learn: 1.1212410	total: 111ms	remaining: 14.2s
40:	learn: 1.1145551	total: 156ms	remaining: 15s
50:	learn: 1.1115610	total: 191ms	remaining: 14.8s
60:	learn: 1.1098745	total: 234ms	remaining: 15.1s
70:	learn: 1.1085350	total: 269ms	remaining: 14.9s
80:	learn: 1.1075906	total: 309ms	remaining: 15s
90:	learn: 1.1066313	total: 346ms	remaining: 14.9s
100:	learn: 1.1057821	total: 381ms	remaining: 14.7s
110:	learn: 1.1050293	total: 416ms	remaining: 14.6s
120:	learn: 1.1043215	total: 459ms	remaining: 14.7s
130:	learn: 1.1037031	total: 494ms	remaining: 14.6s
140:	learn: 1.1032828	total: 526ms	remaining: 14.4s
150:	learn: 1.1027404	total: 559ms	remaining: 14.3s
160:	learn: 1.1022827	total: 596ms	remaining: 14.2s
170:	learn: 1.1018391	total: 641ms	remaining: 14.3s
180:	learn: 1.1013502	total: 674ms	remaining: 14.2s
190:	learn: 1.1008491	to

0:	learn: 1.3525827	total: 5.09ms	remaining: 20.4s
10:	learn: 1.1883195	total: 43ms	remaining: 15.6s
20:	learn: 1.1390094	total: 84.9ms	remaining: 16.1s
30:	learn: 1.1213932	total: 124ms	remaining: 15.9s
40:	learn: 1.1146038	total: 177ms	remaining: 17s
50:	learn: 1.1115327	total: 222ms	remaining: 17.2s
60:	learn: 1.1097942	total: 258ms	remaining: 16.7s
70:	learn: 1.1084994	total: 301ms	remaining: 16.6s
80:	learn: 1.1074423	total: 339ms	remaining: 16.4s
90:	learn: 1.1064931	total: 376ms	remaining: 16.2s
100:	learn: 1.1055630	total: 417ms	remaining: 16.1s
110:	learn: 1.1049411	total: 475ms	remaining: 16.6s
120:	learn: 1.1042893	total: 513ms	remaining: 16.4s
130:	learn: 1.1036408	total: 550ms	remaining: 16.2s
140:	learn: 1.1032687	total: 584ms	remaining: 16s
150:	learn: 1.1030505	total: 615ms	remaining: 15.7s
160:	learn: 1.1026358	total: 657ms	remaining: 15.7s
170:	learn: 1.1021997	total: 701ms	remaining: 15.7s
180:	learn: 1.1016886	total: 738ms	remaining: 15.6s
190:	learn: 1.1011196	tota

0:	learn: 1.3525924	total: 5.13ms	remaining: 20.5s
10:	learn: 1.1883053	total: 43.2ms	remaining: 15.7s
20:	learn: 1.1386697	total: 81.3ms	remaining: 15.4s
30:	learn: 1.1210482	total: 119ms	remaining: 15.2s
40:	learn: 1.1142343	total: 166ms	remaining: 16s
50:	learn: 1.1112076	total: 212ms	remaining: 16.4s
60:	learn: 1.1095317	total: 252ms	remaining: 16.3s
70:	learn: 1.1081351	total: 291ms	remaining: 16.1s
80:	learn: 1.1072236	total: 329ms	remaining: 15.9s
90:	learn: 1.1062715	total: 369ms	remaining: 15.8s
100:	learn: 1.1054617	total: 407ms	remaining: 15.7s
110:	learn: 1.1046993	total: 456ms	remaining: 16s
120:	learn: 1.1040842	total: 494ms	remaining: 15.8s
130:	learn: 1.1035548	total: 529ms	remaining: 15.6s
140:	learn: 1.1031537	total: 565ms	remaining: 15.5s
150:	learn: 1.1027962	total: 601ms	remaining: 15.3s
160:	learn: 1.1024425	total: 645ms	remaining: 15.4s
170:	learn: 1.1020255	total: 686ms	remaining: 15.4s
180:	learn: 1.1014944	total: 721ms	remaining: 15.2s
190:	learn: 1.1009743	to

0:	learn: 1.3525773	total: 5.04ms	remaining: 20.2s
10:	learn: 1.1884321	total: 42.9ms	remaining: 15.6s
20:	learn: 1.1388612	total: 80.6ms	remaining: 15.3s
30:	learn: 1.1213686	total: 119ms	remaining: 15.2s
40:	learn: 1.1145232	total: 171ms	remaining: 16.5s
50:	learn: 1.1113502	total: 219ms	remaining: 16.9s
60:	learn: 1.1096784	total: 255ms	remaining: 16.5s
70:	learn: 1.1085103	total: 291ms	remaining: 16.1s
80:	learn: 1.1075162	total: 333ms	remaining: 16.1s
90:	learn: 1.1066340	total: 370ms	remaining: 15.9s
100:	learn: 1.1057793	total: 411ms	remaining: 15.9s
110:	learn: 1.1052044	total: 456ms	remaining: 16s
120:	learn: 1.1046604	total: 503ms	remaining: 16.1s
130:	learn: 1.1039945	total: 539ms	remaining: 15.9s
140:	learn: 1.1035062	total: 575ms	remaining: 15.7s
150:	learn: 1.1029783	total: 612ms	remaining: 15.6s
160:	learn: 1.1025572	total: 647ms	remaining: 15.4s
170:	learn: 1.1020543	total: 699ms	remaining: 15.6s
180:	learn: 1.1015182	total: 747ms	remaining: 15.8s
190:	learn: 1.1011226	

0:	learn: 1.3525778	total: 5.1ms	remaining: 20.4s
10:	learn: 1.1883453	total: 43.1ms	remaining: 15.6s
20:	learn: 1.1387900	total: 82.5ms	remaining: 15.6s
30:	learn: 1.1212146	total: 120ms	remaining: 15.4s
40:	learn: 1.1145156	total: 166ms	remaining: 16.1s
50:	learn: 1.1114850	total: 221ms	remaining: 17.1s
60:	learn: 1.1098458	total: 259ms	remaining: 16.7s
70:	learn: 1.1086052	total: 296ms	remaining: 16.4s
80:	learn: 1.1075598	total: 334ms	remaining: 16.2s
90:	learn: 1.1066566	total: 371ms	remaining: 16s
100:	learn: 1.1058283	total: 409ms	remaining: 15.8s
110:	learn: 1.1053460	total: 454ms	remaining: 15.9s
120:	learn: 1.1046936	total: 496ms	remaining: 15.9s
130:	learn: 1.1042087	total: 532ms	remaining: 15.7s
140:	learn: 1.1037861	total: 569ms	remaining: 15.6s
150:	learn: 1.1031712	total: 604ms	remaining: 15.4s
160:	learn: 1.1028783	total: 637ms	remaining: 15.2s
170:	learn: 1.1024407	total: 683ms	remaining: 15.3s
180:	learn: 1.1018628	total: 720ms	remaining: 15.2s
190:	learn: 1.1014078	t

In [18]:
print(f"Out of fold log loss {metrics.log_loss(target, cb_oof)}")

Out of fold log loss 1.0903385608051261


In [ ]:
cb_estimators[0]

### XGBoost

In [ ]:
def objective(trial):
    data = get_input("train.csv")
    data.drop(columns='id', inplace=True)
    mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
    data['target'] = data['target'].map(mapper)
    target = data['target']
    data = data.drop(columns=['target'])

    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.35)
    dtrain = xgb.Dataset(train_x, label=train_y)
    dvalid = xgb.Dataset(valid_x, label=valid_y)

    param = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        "verbosity": -1,
        'num_class': 4,
        'max_depth': trial.suggest_int('max_depth', 3,7),
        'learning_rate': trial.suggest_uniform('learning_rate', 0, 0.13 ),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "multi_logloss")
    gbm = lgb.train(
        param, dtrain, valid_sets=[dvalid], verbose_eval=False, callbacks=[pruning_callback]
    )

    preds = gbm.predict(valid_x)
    log_loss = sklearn.metrics.log_loss(valid_y, preds)
    return log_loss

In [ ]:
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=20), direction="minimize"
    )
study.optimize(objective, n_trials=200)

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

In [ ]:
dtrain = xgb.DMatrix(
    data=data, label=target
)

xgb_params = {
    "booster": "gbtree",
    "eta": "0.1",
    "max_depth": 3,
    "random_seed": 42,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'random_state': 13,
    'learning_rate': 0.08356451010151393, 
    'gamma': 0.02911685058980812, 
    'max_depth': 3, 
    'min_child_weight': 10.748514454096288, 
    'max_delta_step': 2.4474818433727927, 
    'subsample': 0.6445037550866027, 
    'colsample_bytree': 0.07634753656242108, 
    'lambda': 13.663280761461781, 
    'alpha': 21.521205761694137, 
    'max_leaves': 48,
    'num_class': 4,
    'tree_method': 'gpu_hist',
}

cv_result_xgb = xgb.cv(
    params=xgb_params,
    dtrain=dtrain,
    num_boost_round=2000,
    #categorical_feature=categorical_feature_names,
    early_stopping_rounds=100,
    verbose_eval=10,
    stratified=True,
    seed=42,
    metrics="mlogloss",
    shuffle=True,
    nfold=5,
    #maximaze=True
)

In [ ]:
cv_result_xgb.shape[0]

1838

In [9]:
import lightgbm as lgb
data = get_input("train.csv")
test = get_input("test.csv")

data.drop(columns='id', inplace=True)
mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
data['target'] = data['target'].map(mapper)

target = data['target']
data = data.drop(columns=['target'])

train.csv: shape = 100000 rows, 52 cols
test.csv: shape = 50000 rows, 51 cols


In [10]:
xgb_params = {
    "booster": "gbtree",
    #"eta": 0.1,
    "random_seed": 42,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'random_state': 13,
    'learning_rate': 0.08, 
    'gamma': 0.02911685058980812, 
    'max_depth': 3,
    'min_child_weight': 10.748514454096288, 
    'max_delta_step': 2.4474818433727927,
    'subsample': 0.6445037550866027,
    'colsample_bytree': 0.07,
    #'colsample_bylevel': 0.7,
    #'colsample_bynode': 0.7,
    'reg_lambda': 1, 
    'reg_alpha': 5, 
    'num_class': 4,
    'n_estimators': 710,  # надо потюнить
    'tree_method': 'gpu_hist',
} # 1.09042

xgb_params = {
    "booster": "gbtree",
    #"eta": 0.1,
    "random_seed": 42,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'random_state': 13,
    'learning_rate': 0.08, 
    'gamma': 0.02911685058980812, 
    'max_depth': 3,
    'min_child_weight': 10.748514454096288, 
    'max_delta_step': 2.4474818433727927,
    'subsample': 0.6445037550866027,
    'colsample_bytree': 0.07,
    #'colsample_bylevel': 0.7,
    #'colsample_bynode': 0.7,
    'reg_lambda': 1, 
    'reg_alpha': 5, 
    'num_class': 4,
    'n_estimators': 850,  # надо потюнить
    'tree_method': 'gpu_hist',
}

In [11]:
cv = StratifiedKFold(n_splits=15, random_state=435, shuffle=True)

xgb_estimators, xgb_encoders, xgb_oof = xgboost_cv_fit(
    xgb_params, data, target, cv, #categorical=categorical_feature_names
)

Sun May 30 13:18:10 2021, Cross-Validation, 100000 rows, 50 cols
Fold 1, Valid score = 1.09484
Fold 2, Valid score = 1.08842
Fold 3, Valid score = 1.08464
Fold 4, Valid score = 1.09311
Fold 5, Valid score = 1.0897
Fold 6, Valid score = 1.09275
Fold 7, Valid score = 1.09376
Fold 8, Valid score = 1.08843
Fold 9, Valid score = 1.08901
Fold 10, Valid score = 1.09406
Fold 11, Valid score = 1.0894
Fold 12, Valid score = 1.09054
Fold 13, Valid score = 1.09157
Fold 14, Valid score = 1.08504
Fold 15, Valid score = 1.08761


In [12]:
print(f"Out of fold log loss {metrics.log_loss(target, xgb_oof)}")

Out of fold log loss 1.0901912643016558


In [ ]:
xgb_estimators[0]

XGBClassifier(alpha=21.521205761694137, colsample_bytree=0.07634753656242108,
              eta=0.1, eval_metric='mlogloss', gamma=0.02911685058980812,
              lambda=13.663280761461781, learning_rate=0.08356451010151393,
              max_delta_step=2.4474818433727927, max_leaves=48,
              min_child_weight=10.748514454096288, n_estimators=700,
              num_class=4, objective='multi:softprob', random_seed=42,
              random_state=13, subsample=0.6445037550866027,
              tree_method='gpu_hist')

Проверим как работает усреднение

In [38]:
result_lgb_xgb_cb = np.dstack((lgb_oof, cb_oof, xgb_oof))
result_lgb_xgb_cb_mean = np.mean(result_lgb_xgb_cb, axis=2)
print(f"Out of fold log loss {metrics.log_loss(target, result_lgb_xgb_cb_mean)}")

Out of fold log loss 1.0898707403357817


### Модель второго уровня - логистическая регрессия

In [19]:
result_lgbm_df = pd.DataFrame(lgb_oof)
result_xgb_df = pd.DataFrame(xgb_oof)
result_cb_df = pd.DataFrame(cb_oof)
result_lgbm_df.rename(columns={0:'lgb_class_1',	1:'lgb_class_2',	2:'lgb_class_3',	3:'lgb_class_4'}, inplace=True)
result_xgb_df.rename(columns={0:'xgb_class_1',	1:'xgb_class_2',	2:'xgb_class_3',	3:'xgb_class_4'}, inplace=True)
result_cb_df.rename(columns={0:'cb_class_1',	1:'cb_class_2',	2:'cb_class_3',	3:'cb_class_4'}, inplace=True)
result_valid = pd.concat([result_lgbm_df, result_xgb_df, result_cb_df], axis=1) 

In [20]:
x_train_logreg, x_valid_logreg, y_train_logreg, y_valid_logreg = train_test_split(
    result_valid, target, train_size=0.7, shuffle=True, random_state=30,
)

params = {'random_state': 0, 
          'max_iter': 300, 
          'multi_class': 'multinomial', 
          'solver': 'lbfgs'}

logreg = LogisticRegression(**params)
logreg.fit(x_train_logreg, y_train_logreg)

# 1.08521
y_pred_logreg = logreg.predict_proba(x_valid_logreg)
metrics.log_loss(y_valid_logreg, y_pred_logreg)

1.0847737572719598

In [21]:
# Обучаем на всем наборе 1.08522
logreg = LogisticRegression(**params)
logreg.fit(result_valid, target)

LogisticRegression(max_iter=300, multi_class='multinomial', random_state=0)

### Модель второго уровня - ансамбль логистических регрессий

In [48]:
logreg_params = {'random_state': 0, 
          'max_iter': 500, 
          'multi_class': 'multinomial', 
          'solver': 'lbfgs',
          'C':5
          }

cv = StratifiedKFold(n_splits=10, random_state=435, shuffle=True)
estimators_log_reg, oof_log_reg, scalers = logistic_cv_fit(logreg_params, result_valid, target, cv)

Sun May 30 14:24:26 2021, Cross-Validation, 100000 rows, 12 cols
Fold 1, Valid score = 1.09193
Fold 2, Valid score = 1.0852
Fold 3, Valid score = 1.08927
Fold 4, Valid score = 1.09131
Fold 5, Valid score = 1.08915
Fold 6, Valid score = 1.08704
Fold 7, Valid score = 1.088
Fold 8, Valid score = 1.09155
Fold 9, Valid score = 1.08871
Fold 10, Valid score = 1.08573
Score by each fold: [1.09193, 1.0852, 1.08927, 1.09131, 1.08915, 1.08704, 1.088, 1.09155, 1.08871, 1.08573]


In [49]:
# 1.08522
print(f"Out of fold log loss {metrics.log_loss(target, oof_log_reg)}")

Out of fold log loss 1.0887875510703202


## Получение результата (одна модель логистической регрессии)

In [24]:
result_xgb_test = multi_estimators_predict(xgb_estimators, test.drop(columns='id'))
result_lgb_test = multi_estimators_predict(lgb_estimators, test.drop(columns='id'))
result_cb_test = multi_estimators_predict(cb_estimators, test.drop(columns='id'))

result_lgb_test_df = pd.DataFrame(result_lgb_test)
result_xgb_test_df = pd.DataFrame(result_xgb_test)
result_cb_test_df = pd.DataFrame(result_cb_test)
result_lgb_test_df.rename(columns={0:'lgb_class_1',	1:'lgb_class_2',	2:'lgb_class_3',	3:'lgb_class_4'}, inplace=True)
result_xgb_test_df.rename(columns={0:'xgb_class_1',	1:'xgb_class_2',	2:'xgb_class_3',	3:'xgb_class_4'}, inplace=True)
result_cb_test_df.rename(columns={0:'cb_class_1',	1:'cb_class_2',	2:'cb_class_3',	3:'cb_class_4'}, inplace=True)
result_for_logreg = pd.concat([result_lgb_test_df, result_xgb_test_df, result_cb_test_df], axis=1) 

y_pred_logreg = logreg.predict_proba(result_for_logreg)
test_pred_df = pd.DataFrame(y_pred_logreg)
test_pred_df.rename(columns={0:'class_1',	1:'class_2',	2:'class_3',	3:'class_4'}, inplace=True)
result = test.drop(columns = test.columns.to_list()[1:])
result = pd.concat([result, test_pred_df], axis=1)
result.to_csv('result.csv', index=False)

## Получение результата (одна модель логистической регрессии) без усреднения бустинговых моделей

In [109]:
def multi_estimators_predict_1(estimators: List,
                             X: pd.DataFrame):
  
  cv = KFold(n_splits=len(estimators), random_state=435, shuffle=True)
  preds = np.zeros((X.shape[0], 4))

  for fold, (valid_idx, g) in enumerate(cv.split(X, X['feature_0'])):
    preds[valid_idx] = estimators[fold].predict_proba(X.loc[valid_idx])

  return preds

In [90]:
test.reset_index(inplace=True, drop=True)

In [110]:
result_xgb_test = multi_estimators_predict_1(xgb_estimators, test.drop(columns='id'))
result_lgb_test = multi_estimators_predict_1(lgb_estimators, test.drop(columns='id'))
result_cb_test = multi_estimators_predict_1(cb_estimators, test.drop(columns='id'))

result_lgb_test_df = pd.DataFrame(result_lgb_test)
result_xgb_test_df = pd.DataFrame(result_xgb_test)
result_cb_test_df = pd.DataFrame(result_cb_test)
result_lgb_test_df.rename(columns={0:'lgb_class_1',	1:'lgb_class_2',	2:'lgb_class_3',	3:'lgb_class_4'}, inplace=True)
result_xgb_test_df.rename(columns={0:'xgb_class_1',	1:'xgb_class_2',	2:'xgb_class_3',	3:'xgb_class_4'}, inplace=True)
result_cb_test_df.rename(columns={0:'cb_class_1',	1:'cb_class_2',	2:'cb_class_3',	3:'cb_class_4'}, inplace=True)
result_for_logreg = pd.concat([result_lgb_test_df, result_xgb_test_df, result_cb_test_df], axis=1) 

y_pred_logreg = logreg.predict_proba(result_for_logreg)
test_pred_df = pd.DataFrame(y_pred_logreg)
test_pred_df.rename(columns={0:'class_1',	1:'class_2',	2:'class_3',	3:'class_4'}, inplace=True)
result = test.drop(columns = test.columns.to_list()[1:])
result = pd.concat([result, test_pred_df], axis=1)
result.to_csv('result.csv', index=False)

Сработало плохо

## Получение результата (ансамбль логистических регрессий)

In [27]:
result_xgb_test = multi_estimators_predict(xgb_estimators, test.drop(columns='id'))
result_lgb_test = multi_estimators_predict(lgb_estimators, test.drop(columns='id'))
result_cb_test = multi_estimators_predict(cb_estimators, test.drop(columns='id'))

result_lgb_test_df = pd.DataFrame(result_lgb_test)
result_xgb_test_df = pd.DataFrame(result_xgb_test)
result_cb_test_df = pd.DataFrame(result_cb_test)
result_lgb_test_df.rename(columns={0:'lgb_class_1',	1:'lgb_class_2',	2:'lgb_class_3',	3:'lgb_class_4'}, inplace=True)
result_xgb_test_df.rename(columns={0:'xgb_class_1',	1:'xgb_class_2',	2:'xgb_class_3',	3:'xgb_class_4'}, inplace=True)
result_cb_test_df.rename(columns={0:'cb_class_1',	1:'cb_class_2',	2:'cb_class_3',	3:'cb_class_4'}, inplace=True)
result_for_logreg = pd.concat([result_lgb_test_df, result_xgb_test_df, result_cb_test_df], axis=1) 

In [28]:
result = multi_estimators_predict(estimators_log_reg, result_for_logreg)

In [29]:
test_pred_df = pd.DataFrame(result)
test_pred_df.rename(columns={0:'class_1',	1:'class_2',	2:'class_3',	3:'class_4'}, inplace=True)
result = test.drop(columns = test.columns.to_list()[1:])
result = pd.concat([result, test_pred_df], axis=1)
result.to_csv('result.csv', index=False)

С предыдущим вариантом различий нет, так как линейные модели очень похожи друг на друга

## Получение результата (одна модель логистической регрессии) с предсказаниями лучших бустинговых моделей

In [43]:
result_cb_test = cb_estimators[2].predict_proba(test.drop(columns='id'))
result_lgb_test = lgb_estimators[2].predict_proba(test.drop(columns='id'))
result_xgb_test = xgb_estimators[10].predict_proba(test.drop(columns='id'))

result_lgb_test_df = pd.DataFrame(result_lgb_test)
result_xgb_test_df = pd.DataFrame(result_xgb_test)
result_cb_test_df = pd.DataFrame(result_cb_test)
result_lgb_test_df.rename(columns={0:'lgb_class_1',	1:'lgb_class_2',	2:'lgb_class_3',	3:'lgb_class_4'}, inplace=True)
result_xgb_test_df.rename(columns={0:'xgb_class_1',	1:'xgb_class_2',	2:'xgb_class_3',	3:'xgb_class_4'}, inplace=True)
result_cb_test_df.rename(columns={0:'cb_class_1',	1:'cb_class_2',	2:'cb_class_3',	3:'cb_class_4'}, inplace=True)
result_for_logreg = pd.concat([result_lgb_test_df, result_xgb_test_df, result_cb_test_df], axis=1) 

In [44]:
y_pred_logreg = logreg.predict_proba(result_for_logreg)
test_pred_df = pd.DataFrame(y_pred_logreg)
test_pred_df.rename(columns={0:'class_1',	1:'class_2',	2:'class_3',	3:'class_4'}, inplace=True)
result = test.drop(columns = test.columns.to_list()[1:])
result = pd.concat([result, test_pred_df], axis=1)
result.to_csv('result.csv', index=False)

## Получение результата (одна модель логистической регрессии) с предсказаниями бустинговых моделей, заново обученных на исходном датасете

In [54]:
lgb_params = {
    "boosting_type ": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    'n_estimators': 710,
    'num_class': 4,
    'lambda_l1': 0.02,
    'lambda_l2': 150,
    'num_leaves': 7,
    'feature_fraction': 0.11,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    'min_child_samples': 80,
    'max_depth': 5,
    'learning_rate': 0.075,
    'random_state': 42,
    #'max_bin': 50,
    #'device' : 'gpu'
}

cb_params = {
    "n_estimators": 4000,
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    "task_type": "GPU",
    'min_data_in_leaf': 35,
    'depth': 3,
    'learning_rate': 0.06,
    'random_strength' : 10,
    "l2_leaf_reg": 300,                       
    'grow_policy': "SymmetricTree",
    "random_seed": 27,
    "classes_count": 4,
    'bootstrap_type':  'Bayesian',
    #'bagging_temperature': 50, #for Bayesian bootstrap_type
    #'subsample': 0.6 # for Poisson, Bernoulli, MVS bootstrap_type
}

xgb_params = {
    "booster": "gbtree",
    #"eta": 0.1,
    "random_seed": 42,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'random_state': 13,
    'learning_rate': 0.08, 
    'gamma': 0.02911685058980812, 
    'max_depth': 3,
    'min_child_weight': 10.748514454096288, 
    'max_delta_step': 2.4474818433727927,
    'subsample': 0.6445037550866027,
    'colsample_bytree': 0.07,
    #'colsample_bylevel': 0.7,
    #'colsample_bynode': 0.7,
    'reg_lambda': 1, 
    'reg_alpha': 5, 
    'num_class': 4,
    'n_estimators': 850,  # надо потюнить
    'tree_method': 'gpu_hist',
}

In [ ]:
model_catb = cb.CatBoostClassifier(**cb_params)
model_catb.fit(data, target, verbose=10)

model_lgb = lgb.LGBMClassifier(**lgb_params)
model_lgb.fit(data, target, verbose=10)

model_xgb = xgb.XGBClassifier(**xgb_params)
model_xgb.fit(data, target, verbose=10)

result_cb_test_1 = model_catb.predict_proba(test.drop(columns='id'))
result_lgb_test_1 = model_lgb.predict_proba(test.drop(columns='id'))
result_xgb_test_1 = model_xgb.predict_proba(test.drop(columns='id'))

In [58]:
result_lgb_test_df = pd.DataFrame(result_lgb_test_1)
result_xgb_test_df = pd.DataFrame(result_xgb_test_1)
result_cb_test_df = pd.DataFrame(result_cb_test_1)
result_lgb_test_df.rename(columns={0:'lgb_class_1',	1:'lgb_class_2',	2:'lgb_class_3',	3:'lgb_class_4'}, inplace=True)
result_xgb_test_df.rename(columns={0:'xgb_class_1',	1:'xgb_class_2',	2:'xgb_class_3',	3:'xgb_class_4'}, inplace=True)
result_cb_test_df.rename(columns={0:'cb_class_1',	1:'cb_class_2',	2:'cb_class_3',	3:'cb_class_4'}, inplace=True)
result_for_logreg = pd.concat([result_lgb_test_df, result_xgb_test_df, result_cb_test_df], axis=1) 

In [59]:
y_pred_logreg = logreg.predict_proba(result_for_logreg)
test_pred_df = pd.DataFrame(y_pred_logreg)
test_pred_df.rename(columns={0:'class_1',	1:'class_2',	2:'class_3',	3:'class_4'}, inplace=True)
result = test.drop(columns = test.columns.to_list()[1:])
result = pd.concat([result, test_pred_df], axis=1)
result.to_csv('result.csv', index=False)

1.08558 видимо переобучать заново бустинговые алгоритмы - не лучший выбор